In [1]:
import os
import math
import pandas as pd
import numpy as np
import requests
from requests.auth import HTTPBasicAuth

In [39]:
def data_merge(List = pd.DataFrame(), raw_data = {}, export = 'merge'):
    """
        작성자: sjyoo
        작성일: 190612
        기능: list와 raw 파일 COR, ACC, RT 등을 merge
        입력: list = pd.DataFrame(), raw_data = dict{}
        출력: merge.xlsx export
        용례: data_merge(list_raw, data, str(export excel file name))
    """
    tasks = {1:'REST',2:'CBTTF',3:'CBTTB',4:'GNG',5:'TWOBACK',6:'STRC',7:'STRI',8:'VFT',9:'REPEAT',10:'MIND',11:'FOLD'}
    print('merge 시작')
    for k, v in tasks.items():
        print('key : '+ str(k) + ', value : ' + v)
        raw_data[v] = raw_data[v].replace(['TASKMARKER','OUTTOUCH', 'TASKMARKER\r','OUTTOUCH\r'],[np.nan, np.nan, np.nan, np.nan]).T.dropna(axis = 1, how = 'all')
        raw_data[v].fillna('')
        pack = pd.DataFrame()
        pack['uuid'] = raw_data[v].index
        idx = raw_data[v].index
        col = raw_data[v].columns
        
        if 'GNG' in v or 'TWOBACK' in v or 'STRC' in v or 'STRI' in v:
            Cor_list = []
            RT = []
            Acc =[]
            for i in idx:
                Cor = 0
                LV = 0
                RT_list = []
                for j in col:
                    if pd.isnull(raw_data[v][j][i]) == False:
                        marker = raw_data[v][j][i].split('/')
                        if 'LV' in marker[0]:
                            Cor += int(marker[1])
                            LV += 1
                            if '1-1' in marker[2]:
                                if int(marker[3]) < 2000 :
                                    RTint = int(marker[3])
                                    RT_list.append(RTint)
                        else:
                            pass
                    else:
                        pass
                 
                Cor_list.append(Cor)
                if len(RT_list) is not 0:
                    RT.append(sum(RT_list) / len(RT_list))
                else:
                    RT.append(0)
                
                if LV == 0:
                    Acc.append(0)
                else:
                    Acc.append(Cor/LV)
                
            pack['%s_Cor' % v] = Cor_list
            pack['%s_RT' % v] = RT
            pack['%s_ACC' % v] = Acc
            print('정답률, 반응시간')
            print('과제 완료')
            
            
        elif 'CBTTF' in v or 'CBTTB' in v or 'MIND' in v or 'REPEAT' in v or 'FOLD' in v:
            Cor_list = []
            Acc =[]
            for i in idx:
                Cor = 0
                LV = 0
                for j in col:
                    if pd.isnull(raw_data[v][j][i]) == False:
                        marker = raw_data[v][j][i].split('/')
                        print(marker)
                        if 'LV' in marker[0]:
                            Cor += int(marker[1])
                            LV += 1
                        else:
                            pass
                    else:
                        pass
                
                Cor_list.append(Cor)
                if LV == 0:
                    Acc.append(0)
                else:
                    Acc.append(Cor/LV)
            print(pack)
            print(Cor_list)
            pack['%s_Cor' % v] = Cor_list
            pack['%s_ACC' % v] = Acc
            print('정답률')
            print('과제 완료')
            
        else:
            print('pass')
            pass
        
        print(List)
        List = List.merge(pack, on='uuid', how = 'outer')

    List.to_excel(export + '.xlsx', sheet_name = 'data')
    print('caculate 완료')
    return
    
def raw_to_data(raw):
    """
        작성자: sjyoo
        작성일: 190611
        기능: table을 분석용 dict로 내보내기
        입력: raw table(dataframe)
        출력: raw_data = dict{pd.DataFrame{task pivot)}
        용례: raw_data = raw_to_data(pandas DataFrame(raw))
        메모: 데이터에 따라 pivot/pivot_table 조절
    """
    tasks = {1:'REST',2:'CBTTF',3:'CBTTB',4:'GNG',5:'TWOBACK',6:'STRC',7:'STRI',8:'VFT',9:'REPEAT',10:'MIND',11:'FOLD'}
    raw_data = {}
    for task in tasks.values():
        raw_data[task]= pd.DataFrame()
    
    task_num = raw['taskType'].unique()
    for k in task_num:
        task_ind = raw.query('taskType == "' + k +'"')
        # data = task_ind.pivot(columns = 'childName', values = 'raw')
        # data = pd.pivot_table(task_ind, index = 'index', columns = 'childName', values = 'raw', aggfunc=np.sum, margins = True)
        raw_data[tasks[int(k)]] = pd.pivot_table(task_ind, index = 'index', columns = 'taskUUID', values = 'raw', aggfunc=np.sum)
        print('%s  실행' % k)
    return raw_data

def request_to_table(url, uuid, my_id, my_pw):
    """
        작성자: cmlee
        수정자: sjyoo
        작성일: 190509
        수정일: 190611 
        기능: 요청 정보를 이용해서 서버에서 데이터 테이블 형태로 반환
        입력: url, id, pw, taskuuid
        출력: pandas DataFrame : header + contents
        용례: raw, list_raw = request_to_table(str(api_url), str(uuid)/ list(uuid), str(my_id), str(my_pw))
    """
    raw = pd.DataFrame()
    list_raw = pd.DataFrame()
    print('table 요청 시작')
    for child in uuid:
        print(child + '_로딩 중....')
        url_uuid = url + '?taskUUID=' + child
        r = requests.get(url=url_uuid, auth=HTTPBasicAuth(my_id, my_pw))
        r = r.content.decode("utf-8")
        r = r.split('\n')
        t_head = r.pop(0).split(',')
        t_head[-1] = t_head[-1][0:-1]
        t_body = [t.split(',') for t in r if t != '']
        full = pd.DataFrame(t_body, columns=t_head)
        full = full[full.taskUUID != '']
        tasks = full['taskType'].unique()
        for task in tasks:
            task_raw = full.query('taskType == "' + str(task) +'"')
            mx = task_raw.taskSubID.max()
            if mx != '':
                t_mx_raw = task_raw.query('taskSubID == "'+str(mx)+'"')
                raw = pd.concat([raw, t_mx_raw], axis=0)
        print('%s 완료' % child)
    list_raw['name'] = raw['childName'].unique()
    list_raw['uuid'] = raw['taskUUID'].unique()
    print("raw export 완료")
    return raw, list_raw

def access_to_table(connect_info, uuid, table = 'uuid'):
    """
        작성자: sjyoo
        작성일: 190611
        기능: table 접속
        입력: connect_info: id, pw; uuid = taskuuid; table = 받고 싶은 데이터 table
        출력: pandas DataFrame : 현재는 performance 
        용례: raw = access_to_table(connect_info, uuid, table = 'uuid'&'raw'&'perf')
    """
    # base_url = 'http://ec2-15-164-48-95.ap-northeast-2.compute.amazonaws.com'
    base_url = 'http://obelab-api.com'
    api_uuid = base_url + '/api/analysis/taskuuid/'
    api_raw = base_url + '/api/analysis/metaraw/'
    api_perf = base_url + '/api/analysis/metamarker/'
    print('table access 시작')
    if table == 'uuid':
        return request_to_table(api_uuid, uuid, connect_info[0], connect_info[1])
    elif table == 'raw':
        return request_to_table(api_raw, uuid, connect_info[0], connect_info[1])
    elif table == 'perf':
        return request_to_table(api_perf, uuid, connect_info[0], connect_info[1])
    print("raw 완료")


def get_UUID(connect_info, date1 = '2000-01-01', date2 = '2030-01-01'):
    """
        작성자: sjyoo
        작성일: 190612
        기능: date로 아이디 뽑아내기. UUID 뽑을 거임.
        입력: 접속 정보, 시작일, 종료일
        출력: uuid 리스트
        용례: get_UUID(접속정보, 시작일, 종료일)
    """
    base_url = 'http://obelab-api.com'
    api_uuid = base_url + '/api/analysis/taskuuid/'

    uuid = pd.DataFrame()
    #지금은 다 불러와서 date로 filtering
    r = requests.get(url=api_uuid, auth=HTTPBasicAuth(connect_info[0], connect_info[1]))
    r = r.content.decode("utf-8")
    r = r.split('\n')
    t_head = r.pop(0).split(',')
    t_head[-1] = t_head[-1][0:-1]
    t_body = [t.split(',') for t in r]
    data = pd.DataFrame(t_body, columns=t_head)
    return data.query('taskCreated >= "' + date1 + '" and taskCreated <= "' + date2 +'"')

def server_connect_info():
    my_id = 'admin'
    my_pw = 'obe1234'

    return my_id, my_pw

In [34]:
connect_info = server_connect_info()
# p_list = get_UUID(connect_info, '2019-06-21', '2019-06-24')
# target_child = p_list.taskUUID
# print(p_list.columns)
target_child = ['HgwQ9iNB', 'vnTJNL39']
print(target_child)

['HgwQ9iNB', 'vnTJNL39']


In [35]:
raw, list_raw = access_to_table(connect_info = connect_info, uuid=target_child, table = 'perf') #한예준

table access 시작
table 요청 시작
HgwQ9iNB_로딩 중....
HgwQ9iNB 완료
vnTJNL39_로딩 중....
vnTJNL39 완료
raw export 완료


In [36]:
print(list_raw)

  name      uuid
0  송보민  HgwQ9iNB
1  김수현  vnTJNL39


In [37]:
raw_data = raw_to_data(raw)

1  실행
2  실행
3  실행
4  실행
5  실행
6  실행
7  실행
8  실행
9  실행
10  실행
11  실행


In [40]:
data_merge(list_raw, raw_data, 'Result_0628')

merge 시작
key : 1, value : REST
pass
  name      uuid
0  송보민  HgwQ9iNB
1  김수현  vnTJNL39
key : 2, value : CBTTF
['LV3', '0', '378-637', '4019\r']
['LV3', '1', '374-374', '3439\r']
['LV3', '0', '871-875', '2849\r']
['LV3', '1', '276-276', '3432\r']
['LV3', '1', '385-385', '2589\r']
['LV4', '1', '7814-7814', '4094\r']
['LV4', '1', '3578-3578', '4353\r']
['LV5', '0', '672-62715', '4508\r']
['LV5', '0', '8-71826', '3972\r']
['LV3', '1', '784-784', '2816\r']
['LV3', '1', '452-452', '2617\r']
['LV4', '1', '6283-6283', '4021\r']
['LV4', '1', '1867-1867', '2740\r']
['LV5', '1', '16283-16283', '3526\r']
['LV5', '0', '23645-23654', '3329\r']
['LV5', '0', '52618-52681', '3265\r']
['LV5', '1', '52417-52417', '3177\r']
['LV5', '1', '43251-43251', '3608\r']
    uuid
0   5338
1   5339
2   5340
3   5341
4   5342
5   5343
6   5344
7   5345
8   5347
9   5351
10  5352
11  5353
12  5354
13  5355
14  5356
15  5357
16  5358
17  5359
[0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1]
정답률
과제 완료
  name      

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import scipy as sp
from scipy import stats
import matplotlib as mpl
import seaborn as sns
from matplotlib import pyplot as plt

path = os.getcwd()
f = os.path.join(path, "data", 'Result_0624_1.xlsx')
List = pd.read_excel(open(f, 'rb'), sheet_name = 'data4', header=0, index_col = 0)


# List[List.dtypes[(List.dtypes=="float64")|(List.dtypes=="int64")]
#                         .index.values].hist(figsize=[15,15])

print(List.columns)
List = List.loc[(List.date == '6월 23일') | (List.date == '6월 22일')]
List['GNG_z'] = List['GNG_ACC'] / List['GNG_RT']*1000
List['STRC_z'] =  List['STRI_ACC'] / List['STRI_RT']*1000
List['STRI_z'] =  List['STRC_ACC'] / List['STRC_RT']*1000
List['STRE_z'] =  List['STRI_z'] - List['STRC_z']

np.percentile(List['STRE_z'], 50)
print(List['REPEAT_ACC'])

In [ ]:
import scipy.stats as st
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm


sns.set(font_scale=0.5)
mpl.rcParams['axes.unicode_minus'] = False
path = 'C:/Windows/Fonts/HYSNRL.ttf'
font_name = fm.FontProperties(fname=path).get_name()
plt.rc('font', family=font_name)

for feature in List:
    print(feature)
    if ['ID', 'num', 'date', 'time', 'Sex', 'name', 'Cor', 'grade'] in feature: 
        print('pass')
        pass
    else:
        a = List[feature].dropna()
        low = a[a < np.percentile(a, 30)].count()
        high = a[a > np.percentile(a, 70)].count()
        middle = a[(a <= np.percentile(a, 70))&(a >= np.percentile(a, 30))].count()
        
        points = sns.distplot(a.dropna() 
#                               , bins = 1000
#                               , fit=sp.stats.norm
                              , hist =False, kde = True, kde_kws = {'shade' :True, 'kernel' : 'gau'
#                                                                  , 'bw' : 0.1
                                                                    , 'clip' :(a.min(),a.max()) 
                                                                }
                             ).get_lines()[0].get_data()
        
        x = points[0]
        y = points[1]
#         th = sms.DescrStatsW(a).tconfint_mean(0.0001)[0]
#         se = sms.DescrStatsW(a).tconfint_mean(0.0001)[1]
        plt.fill_between(x,y, where = x > np.percentile(a, 75), color='r', alpha=0.8, label = '높음')
        plt.fill_between(x,y, where = x <  np.percentile(a, 25), color='k', alpha=0.8, label = '낮음')
        plt.fill_between(x,y, where = (x <=  np.percentile(a, 75)) & (x >= np.percentile(a, 25)), color='y', alpha=0.8, label = '보통')
        hito = plt.hist(a, alpha = 0.5, align  = 'right' )
#         plt.text(.50, .5, '낮음 = %s , 중간 = %s , 보통 = %s' % (low, middle, high), horizontalalignment='center',
#      verticalalignment='center',
#      textcoords = 'axes fraction')
        plt.text(x.min(), hito[0].max(), '낮음 = %s , 보통 = %s, 높음 = %s' % (low, middle, high), style='italic',
        bbox={'facecolor':'red', 'alpha':0.5, 'pad':10})
#         plt.annotate('낮음 = %s' % low, xy = (np.percentile(a, 25),np.percentile(a, 25)), xytext=(0.25, 0.5), textcoords = 'axes fraction', arrowprops = dict(color = 'g', alpha= 0.5))
#         plt.annotate('중간 = %s' % middle, xy = (np.percentile(a, 50),np.percentile(a, 50)), xytext=(0.5, 0.7), textcoords = 'axes fraction', arrowprops = dict(color = 'g', alpha= 0.5))
#         plt.annotate('높음 = %s' % high, xy = (np.percentile(a, 75),np.percentile(a, 75)), xytext=(0.75, 0.5), textcoords = 'axes fraction', arrowprops = dict(color = 'g', alpha= 0.5))

        plt.savefig(feature + 'xline10.png', format = 'png', dpi = 1200)
        plt.show()

In [ ]:
import scipy.stats as st
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm


sns.set(font_scale=0.5)
mpl.rcParams['axes.unicode_minus'] = False
path = 'C:/Windows/Fonts/HYSNRL.ttf'
font_name = fm.FontProperties(fname=path).get_name()
plt.rc('font', family=font_name)
for feature in List:
    print(feature)
    if 'ID' in feature or 'num' in feature or 'date' in feature or 'time' in feature or 'Sex' in feature or 'name' in feature or 'Cor' in feature: 
        print('pass')
        pass
    else:
        for grade in List.grade:
            List1 = List[List.grade == grade ]
            a = List1[feature].dropna()
            low = a[a < np.percentile(a, 30)].count()
            high = a[a > np.percentile(a, 70)].count()
            middle = a[(a <= np.percentile(a, 70))&(a >= np.percentile(a, 30))].count()

            points = sns.distplot(a.dropna() , hist =False, kde = True, kde_kws = {'shade' :True, 'kernel' : 'gau' , 'clip' :(a.min(),a.max())}
                                 ).get_lines()[0].get_data()

            x = points[0]
            y = points[1]
            plt.fill_between(x,y, where = x > np.percentile(a, 75), color='r', alpha=0.8, label = '높음')
            plt.fill_between(x,y, where = x <  np.percentile(a, 25), color='k', alpha=0.8, label = '낮음')
            plt.fill_between(x,y, where = (x <=  np.percentile(a, 75)) & (x >= np.percentile(a, 25)), color='y', alpha=0.8, label = '보통')
            hito = plt.hist(a, alpha = 0.5, align  = 'right' )
            plt.text(x.min(), hito[0].max(), '낮음 = %s , 보통 = %s, 높음 = %s' % (low, middle, high), style='italic',
            bbox={'facecolor':'red', 'alpha':0.5, 'pad':10})
            plt.savefig(srt(grade) +'_grade_' +feature + '_line.png', format = 'png', dpi = 1200)
            plt.show()

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm


sns.set(font_scale=0.5)
mpl.rcParams['axes.unicode_minus'] = False
path = 'C:/Windows/Fonts/HYSNRL.ttf'
font_name = fm.FontProperties(fname=path).get_name()
plt.rc('font', family=font_name)

for feature in List:
    print(feature)
    if 'ID' in feature or 'num' in feature or 'date' in feature or 'time' in feature or 'Sex' in feature or 'name' in feature or 'Cor' in feature: 
        print('pass')
        pass
    else:
        
        plt.subplot(2,1,1)
        plt.title(feature)
        a = List[feature]
        low = a[a < np.percentile(a, 30)].count()
        middle = a[a > np.percentile(a, 70)].count()
        high = a[(a <= np.percentile(a, 70))&(a >= np.percentile(a, 30))].count()
        
        points = sns.distplot(a.dropna()
#                               , fit=sp.stats.norm
                              , hist =False, kde = True, kde_kws = {'shade' :True, 'kernel' : 'gau'
#                                                                  , 'bw' : 0.5
                                                                    , 'clip' :(a.min(),a.max()) 
                                                                }
                             ).get_lines()[0].get_data()
        
        x = points[0].cumsum
        y = points[1]
        plt.fill_between(x,y, where = x >= np.percentile(x, 70), color='r', alpha=0.5)
        plt.fill_between(x,y, where = x <= np.percentile(x, 30), color='r', alpha=0.5)
        plt.fill_between(x,y, where = (x<= np.percentile(x, 70)) & (x >= np.percentile(x, 30)), color='y', alpha=0.5)
#         plt.text(y.max()*.80, x.max()*.25, '낮음 = %s , 중간 = %s , 보통 = %s' % (low, middle, high))
        plt.subplot(2,1,2)
        plt.hist(a.dropna(), histtype="barstacked", align = 'left', stacked = True)
        plt.savefig(feature + 'xline1.png', format = 'png', dpi = 1200)
        plt.show()

In [ ]:
g = sns.FacetGrid(List, row='grade', col='Sex')
g.map(sns.distplot, "FOLD_ACC")
plt.show()

In [ ]:
for feature in List:
    print(feature)
    if 'ID' in feature or 'num' in feature or 'date' in feature or 'time' in feature or 'Sex' in feature or 'name' in feature: 
        print('pass')
        pass
    else:
        sns.jointplot(data=List, x='grade', y= feature , kind='reg', color='g')
        plt.savefig(feature + 'xGrade.png', format = 'png', dpi = 300)
        plt.show()

In [ ]:
for feature in List:
    print(feature)
    if 'ID' in feature or 'num' in feature or 'date' in feature or 'time' in feature or 'Sex' in feature or 'name' in feature: 
        print('pass')
        pass
    else:
        sns.jointplot(data=List, x='grade', y= feature , kind='reg', color='g')
        plt.savefig(feature + 'xGrade.png', format = 'png', dpi = 300)
        plt.show()

In [ ]:
for feature in List:
    print(feature)
    if 'ID' in feature or 'num' in feature or 'date' in feature or 'time' in feature or 'Sex' in feature or 'name' in feature: 
        print('pass')
        pass
    else:
        sns.jointplot(data=List, x='Sex', y= feature , kind='hex', color='g')
        plt.savefig(feature + 'xSex.png', format = 'png', dpi = 300)
        plt.show()

In [ ]:
for feature in List:
    print(feature)
    if 'ID' in feature or 'num' in feature or 'date' in feature or 'time' in feature or 'Sex' in feature or 'name' in feature: 
        print('pass')
        pass
    else:
        x1 = List[List.Sex == 1]['%s' % feature].dropna()
        x2 = List[List.Sex == 2]['%s' % feature].dropna()
        p = sp.stats.ttest_ind(x1, x2, equal_var=False)[1]
        sns.distplot(x1, kde=False, fit=sp.stats.norm)
        sns.distplot(x2, kde=False, fit=sp.stats.norm)
        plt.savefig(feature + 'xSex_ttest.png', format = 'png', dpi = 300)
        plt.show()

In [ ]:
print(List.columns)
List1 = List[['Sex', 'grade', 'CBTTF_ACC', 'CBTTB_ACC', 'TWOBACK_RT', 'TWOBACK_ACC', 'Stroop effect',
       'REPEAT_ACC', 'MIND_ACC', 'FOLD_ACC', 'GNG_z', 'STRC_z', 'STRI_z', 'STRE_z']]

sns.heatmap(List1.corr(), annot=True, fmt=".1f" )
plt.savefig('corr_heatmap.png', format='png', dpi=1200) 
plt.show()


In [ ]:
List['grade'].value_counts()

In [ ]:
print(pd.crosstab(List['grade'], List['Sex'], margins = True))

In [ ]:
List.columns

In [ ]:
path = os.getcwd()
f1 = os.path.join(path, "data", 'K-WISC-V_190618.xlsx')
k_wisc = pd.read_excel(open(f1, 'rb'), sheet_name = 'may', header=0, index_col = 0)
f2 = os.path.join(path, "data", 'Result_0624_1.xlsx')
List3 = pd.read_excel(open(f2, 'rb'), sheet_name = 'data', header=0, index_col = 0)
list_k = List3.merge(k_wisc, on = 'name', how = 'inner')

In [ ]:

# list_k.to_excel('list_k.xlsx', sheet_name = 'data')
list_k['GNG_z'] = list_k['GNG_ACC'] / list_k['GNG_RT']*1000
list_k['STRC_z'] =  list_k['STRI_ACC'] / list_k['STRI_RT']*1000
list_k['STRI_z'] =  list_k['STRC_ACC'] / list_k['STRC_RT']*1000

list_k.columns

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

sns.set(font_scale=0.5)
mpl.rcParams['axes.unicode_minus'] = False
path = 'C:/Windows/Fonts/HYSNRL.ttf'
font_name = fm.FontProperties(fname=path).get_name()
plt.rc('font', family=font_name)
path = os.getcwd()
f1 = os.path.join(path, 'list_kwisc.xlsx')
list_k = pd.read_excel(open(f1, 'rb'), header=0, index_col = 0)

list_k1 = list_k[['ID', 'grade', 'sex', 'birth', 'age', 'name', 
       'CBTTF_ACC', 'CBTTB_ACC', 'GNG_RT', 'GNG_ACC',
       'TWOBACK_RT', 'TWOBACK_ACC', 'STRC_RT',
       'STRC_ACC', 'STRI_RT', 'STRI_ACC', 'GNG_z', 'STRC_z', 'STRI_z',
       'REPEAT_ACC', 'MIND_ACC', 'FOLD_ACC', 
       '토막짜기', '공통성', '행렬추리', '숫자', '기호쓰기', '어휘', '무게비교'
       ]]

sns.heatmap(list_k1.corr(), annot=True, fmt=".1f")
plt.savefig('corr.png', format='png', dpi=1200) 
plt.show()





In [ ]:
list_k_1 = list_k[['grade', 'sex', 
       'CBTTF_ACC', 'CBTTB_ACC', 'GNG_RT', 'GNG_ACC',
       'TWOBACK_RT', 'TWOBACK_ACC', 'STRC_RT',
       'STRC_ACC', 'STRI_RT', 'STRI_ACC', 'GNG_z', 'STRC_z', 'STRI_z',
       'REPEAT_ACC', 'MIND_ACC', 'FOLD_ACC', 
       ]]
list_k_2 = list_k[['grade', 
       '토막짜기', '공통성', '행렬추리', '숫자', '기호쓰기', '어휘', '무게비교'
       ]]

# sns.pairplot(list_k_1, hue='grade', size=2.5)
# plt.savefig('list1.png', format='png', dpi=1200) 
# sns.pairplot(list_k_2, hue='grade', size=2.5)
# plt.savefig('list2.png', format='png', dpi=1200) 
# plt.show()

In [ ]:
MIND_per = List.MIND_ACC.rank(pct=True)
MIND_per = MIND_per.sort_values()
mind_cum = np.cumsum(MIND_per/40)
print(List.MIND_ACC.describe())
# plt.xlim(0, 100)
sns.distplot(List.MIND_ACC*100, color = 'y', kde=True, rug=False, hist = False,  
             kde_kws={'cumulative': True, 'bw' : 3, 'shade' : True  })
[line.set_linestyle("--") for line in ax.lines] 
plt.savefig(feature + '_test.png', format = 'png', dpi = 300)
plt.show()
# for feature in List:
#     print(feature)
#     if 'ID' in feature or 'num' in feature or 'date' in feature or 'time' in feature or 'Sex' in feature or 'name' in feature: 
#         print('pass')
#         pass
#     else:
#         x1 = List[List.Sex == 1]['%s' % feature].dropna()
#         x2 = List[List.Sex == 2]['%s' % feature].dropna()
#         p = sp.stats.ttest_ind(x1, x2, equal_var=False)[1]
#         sns.distplot(x1, kde=False, fit=sp.stats.norm)
#         sns.distplot(x2, kde=False, fit=sp.stats.norm)
#         plt.savefig(feature + '_test.png', format = 'png', dpi = 300)
#         plt.show()

In [ ]:
for feature in list_k_2:
    print(feature)
    if 'ID' in feature or 'num' in feature or 'date' in feature or 'time' in feature or 'Sex' in feature or 'name' in feature: 
        print('pass')
        pass
    else:
        sns.jointplot(data=list_k_2, x='grade', y= feature , kind='reg', color='g')
        plt.savefig(feature + 'xGrade.png', format = 'png', dpi = 300)
        plt.show()

In [ ]:
path = os.getcwd()
f = os.path.join(path, "data", '20190620_Data_fNIRS.csv')
fNIRS = pd.read_csv(open(f, 'rb'), header=0)
print(fNIRS)

In [ ]:
fNIRS.sort_values(by = 'fnirs_task_name'  )

In [ ]:
fNIRS_kwisc = fNIRS[fNIRS.fnirs_task_name == 'CBTTB'].merge(k_wisc, left_on = 'fnirs_childName', right_on = 'name', how = 'outer')

In [ ]:
print(fNIRS_kwisc)

In [ ]:
# fNIRS_kwisc.to_excel('fNIRS_kwisc.xlsx', sheet_name = 'data')

In [ ]:
print(List1)

In [ ]:
for feature in List1:
    print(feature)
    if 'ID' in feature or 'num' in feature or 'date' in feature or 'time' in feature or 'Sex' in feature or 'name' in feature or 'TWOBACK_RT' in feature or 'TWOBACK_ACC' in feature: 
        print('pass')
        pass
    else:
        List1['rank_%s'% feature] = List1['%s' % feature].rank()
        z =  np.exp(List1['rank_%s'% feature])
        points = sns.distplot(z.dropna(), kde = True, kde_kws = {'shade' :True, 'bw' : 1}, ).get_lines()[0].get_data()

        x = points[0]
        y = points[1]

        plt.fill_between(x,y, where = x >= x*0.70, color='r')
        plt.fill_between(x,y, where = x <= x*0.30, color='g')
#         plt.fill_between(x,y, where = (x<= x*0.70) & (x>= x*0.30), color='y')
        plt.savefig(feature + 'xline.png', format = 'png', dpi = 300)
        plt.show()

# MIND_per = List.MIND_ACC.rank(pct=True)
# MIND_per = MIND_per.sort_values()
# mind_cum = np.cumsum(MIND_per/40)
# print(List.MIND_ACC.describe())
# # plt.xlim(0, 100)
# sns.distplot(List.MIND_ACC*100, color = 'y', kde=True, rug=False, hist = False,  
#              kde_kws={'cumulative': True, 'bw' : 3, 'shade' : True  })
# [line.set_linestyle("--") for line in ax.lines] 
# plt.savefig(feature + '_test.png', format = 'png', dpi = 300)
# plt.show()